In [3]:
# Import necessary libraries
import ee
import geemap

# Initialize the Earth Engine API
ee.Initialize()

# Convert DataFrame to Earth Engine Feature Collection
soildata_fc = geemap.df_to_ee(soildata_xy, latitude = 'coord_y', longitude = 'coord_x')
# soildata_fc = geemap.df_to_ee(soildata_xy.sample(n=2000), latitude = 'coord_y', longitude = 'coord_x')

# Function to split a feature collection (sampling points) into chunks
def split_sampling_points(fc, chunk_size):
    features = fc.toList(fc.size())
    chunks = [features.slice(i, i + chunk_size) for i in range(0, features.size().getInfo(), chunk_size)]
    return [ee.FeatureCollection(chunk) for chunk in chunks]

# Split the sample points into subsets of 1000 points each
# This is necessary to avoid timeout errors
chunk_size = 1000
soildata_chunks = split_sampling_points(soildata_fc, chunk_size)

In [4]:
# Soil Grids 250m v2.0
# (this takes about 10 minutes to run)

# Soil Grids 250m v2.0: bdod_mean (bulk density)
bdod_image = ee.Image("projects/soilgrids-isric/bdod_mean")
bdod_image = bdod_image.select(['bdod_0-5cm_mean', 'bdod_5-15cm_mean', 'bdod_15-30cm_mean'])

# Soil Grids 250m v2.0: clay_mean (clay content)
clay_image = ee.Image("projects/soilgrids-isric/clay_mean")
clay_image = clay_image.select(['clay_0-5cm_mean', 'clay_5-15cm_mean', 'clay_15-30cm_mean'])

# Soil Grids 250m v2.0: sand_mean (sand content)
sand_image = ee.Image("projects/soilgrids-isric/sand_mean")
sand_image = sand_image.select(['sand_0-5cm_mean', 'sand_5-15cm_mean', 'sand_15-30cm_mean'])

# Soil Grids 250m v2.0: soc_mean (soil organic carbon)
soc_image = ee.Image("projects/soilgrids-isric/soc_mean")
soc_image = soc_image.select(['soc_0-5cm_mean', 'soc_5-15cm_mean', 'soc_15-30cm_mean'])

# Soil Grids 250m v2.0: cfvo_mean (coarse fragments volume)
cfvo_image = ee.Image("projects/soilgrids-isric/cfvo_mean")
cfvo_image = cfvo_image.select(['cfvo_0-5cm_mean', 'cfvo_5-15cm_mean', 'cfvo_15-30cm_mean'])

# Stack the images into a single multiband image
soilgrids_image = bdod_image.addBands([clay_image, sand_image, soc_image, cfvo_image])

# Initialize an empty list to store DataFrames
dataframes = []

# Loop over each subset and sample the data
for chunk in soildata_chunks:
    sampled_points = geemap.extract_values_to_points(chunk, soilgrids_image, scale=250)
    sampled_df = geemap.ee_to_df(sampled_points)
    dataframes.append(sampled_df)

# Concatenate all DataFrames into a single DataFrame
soilgrids_df = pd.concat(dataframes, ignore_index=True)

# Rename columns
# Replace dash with underscores
soilgrids_df.columns = soilgrids_df.columns.str.replace('-', '_')
# Remove _mean from column names
soilgrids_df.columns = soilgrids_df.columns.str.replace('_mean', '')

# Print column names
print(soilgrids_df.columns)

# Check if number of rows of soildata_df is the same as that of soildata_xy
# If the number of rows is the same, the merge was successful
target = soildata_xy.shape[0]
print(
  '\nThere should be', target, 'events:',
  target == soilgrids_df.shape[0], '\nThere are', soilgrids_df.shape[0], 'events')

Index(['bdod_0_5cm', 'bdod_15_30cm', 'bdod_5_15cm', 'cfvo_0_5cm',
       'cfvo_15_30cm', 'cfvo_5_15cm', 'clay_0_5cm', 'clay_15_30cm',
       'clay_5_15cm', 'coord_x', 'coord_y', 'data_coleta_ano', 'dataset_id',
       'observacao_id', 'sand_0_5cm', 'sand_15_30cm', 'sand_5_15cm',
       'soc_0_5cm', 'soc_15_30cm', 'soc_5_15cm'],
      dtype='object')

There should be 11312 events: True 
There are 11312 events


In [5]:
# Compute summary statistics of the SoilGrids data
summary = soilgrids_df.describe()
print(summary)

         bdod_0_5cm  bdod_15_30cm   bdod_5_15cm    cfvo_0_5cm  cfvo_15_30cm  \
count  11004.000000  11004.000000  11004.000000  11022.000000  11022.000000   
mean     118.057343    126.689295    122.025718     62.028216     66.258302   
std       11.322577     10.698621     11.068186     33.820556     37.230729   
min       61.000000     60.000000     60.000000      0.000000      0.000000   
25%      111.000000    123.000000    117.000000     36.000000     38.000000   
50%      119.000000    129.000000    123.000000     56.000000     59.000000   
75%      127.000000    134.000000    130.000000     86.000000     92.000000   
max      145.000000    158.000000    148.000000    364.000000    336.000000   

        cfvo_5_15cm    clay_0_5cm  clay_15_30cm   clay_5_15cm       coord_x  \
count  11022.000000  11022.000000  11022.000000  11022.000000  11312.000000   
mean      63.520504    300.562239    339.032662    303.490383    -53.102308   
std       36.341861    100.885785     96.424365    

In [6]:
from datetime import datetime

# MapBiomas LULC Collection 7.1
# This takes about xx minutes to run

# Import the MapBiomas Collection 7.1
collection = 'projects/mapbiomas-workspace/public/collection7_1/mapbiomas_collection71_integration_v1'
mapbiomas_image = ee.Image(collection)

# Initialize an empty list to store DataFrames
dataframes = []

# Loop over each subset and sample the data
for chunk in soildata_chunks:
    sampled_points = geemap.extract_values_to_points(chunk, mapbiomas_image, scale=30)
    sampled_df = geemap.ee_to_df(sampled_points)
    dataframes.append(sampled_df)

# Concatenate all DataFrames into a single DataFrame
mapbiomas_df = pd.concat(dataframes, ignore_index=True)

# Rename columns
# Remove classification_ prefix from column names
mapbiomas_df.columns = mapbiomas_df.columns.str.replace('classification_', '')

# Get LULC class at the year of sampling (data_coleta_ano)
# Each column in the MapBiomas dataset represents a year. The column name is the year of the
# classification. The value is the class code. We need to extract the class code for the year of
# sampling, which is stored in the data_coleta_ano column.
# Step 1: Create a new column YEAR based on data_coleta_ano
mapbiomas_df['YEAR'] = mapbiomas_df['data_coleta_ano']
# Step 2: If YEAR is less than 1985, set it to 0 (no data)
mapbiomas_df.loc[mapbiomas_df['YEAR'] < 1985, 'YEAR'] = 0
# Step 3: Find the column index for each YEAR
lulc_idx = mapbiomas_df.columns.get_indexer(mapbiomas_df['YEAR'].astype(str))
# Step 4: Extract the class code for each row based on the data_coleta_ano column
lulc = mapbiomas_df.to_numpy()
lulc = lulc[range(len(lulc)), lulc_idx]
# Step 5: Convert the extracted class codes to strings and assign them to a new column lulc
mapbiomas_df['lulc'] = lulc.astype(str)
# Step 6: Drop the YEAR column
mapbiomas_df = mapbiomas_df.drop(columns = ['YEAR'])

# Some columns of mapbiomas_df are named with the year of the classification, ranging from 1985 to
# the present year. This columns need to be dropped.
# Drop columns with years as column names
current_year = datetime.now().year
years_to_drop = [str(year) for year in range(1985, current_year + 1)]
mapbiomas_df.drop(columns=years_to_drop, inplace=True, errors='ignore')

# Reclassify the land cover classes
forest_codes = ['1.0', '3.0', '4.0', '5.0', '49.0']
nonforest_codes = ['10.0', '11.0', '12.0', '32.0', '29.0', '50.0', '13.0']
pasture_codes = ['15.0']
agriculture_codes = ['14.0', '18.0', '19.0', '39.0', '20.0', '40.0', '62.0', '41.0', '36.0', '46.0',
                     '47.0', '48.0', '21.0']
forestry_codes = ['9.0']
nonvegetation_codes = ['22.0', '23.0', '24.0', '30.0', '25.0', '26.0', '33.0', '31.0', '27.0']
unknown_codes = ['0.0', 'nan']
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(forest_codes, 'forest')
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(nonforest_codes, 'nonforest')
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(pasture_codes, 'pasture')
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(agriculture_codes, 'agriculture')
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(forestry_codes, 'forestry')
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(nonvegetation_codes, 'nonvegetation')
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(unknown_codes, 'unknown')

# Print summary of the land cover classes
print('\nDistribution of land cover/land use classes:')
print(mapbiomas_df['lulc'].value_counts())

# Check if number of rows of mapbiomas_df is the same as that of soildata_xy
# If the number of rows is the same, the sampling was successful
target = soildata_xy.shape[0]
print(
  '\nThere should be', target, 'events:',
  target == mapbiomas_df.shape[0], '\nThere are', mapbiomas_df.shape[0], 'events')


Distribution of land cover/land use classes:
lulc
0                4159
forest           2757
pasture          2186
agriculture      1471
nonforest         503
nonvegetation     143
forestry           87
unknown             6
Name: count, dtype: int64

There should be 11312 events: True 
There are 11312 events


In [15]:
# Merge data sampled from SoilGrids and MapBiomas into soildata_df
# Keep all rows from soildata_df
merge_columns = ['dataset_id', 'observacao_id', 'coord_x', 'coord_y', 'data_coleta_ano']
output_df = soildata_df.merge(soilgrids_df, on = merge_columns, how = 'left')
output_df = output_df.merge(mapbiomas_df, on = merge_columns, how = 'left')

# Check if number of rows of output_df is the same as that of soildata_df
# If the number of rows is the same, the merge was successful
target = soildata_df.shape[0]
print(
  '\nThere should be', target, 'events:',
  target == output_df.shape[0], '\nThere are', output_df.shape[0], 'events')


There should be 17606 events: True 
There are 17606 events


In [16]:
# Write the output_df to a TXT file
# Field separator: tab. Decimal separator: comma
file_path = os.path.join(work_dir, 'data', '04a-febr-data.txt')
output_df.to_csv(file_path, sep='\t', decimal=',', index=False)